In [ ]:
#sqlalchemy
#Đây là một pakage cho phép chúng ta kết nối và truy vấn trên những dữ liệu SQL một cách trực tiếp theo mô hình server-client side

#Tên server là gì?
#Tên database trong server cần truy vấn.
#port: Cổng kết nối, thường mặc định của MSSQL là 1443.
#username: Tên user.
#password: Mật khẩu truy cập.

In [ ]:
%%script echo skipping

from sqlalchemy import create_engine, MetaData, Table, select, engine
# Create parameters
TableName = 'WorkOrder'

DB = {
    'drivername': 'mssql+pyodbc',
    'servername': 'LAPTOPTCC-PC',
    #'port': '1443',
    #'username': '',
    #'password': '',
    'database': 'TestDB',
    'driver': 'SQL Server Native Client 11.0',
    'trusted_connection': 'yes',
    'legacy_schema_aliasing': False
}

# Create the connection
engine = create_engine(DB['drivername'] + '://' + DB['servername'] + '/' + DB['database'] + '?' + 'driver=' + DB['driver'] 
+ ';' + 'trusted_connection=' + DB['trusted_connection'], legacy_schema_aliasing=DB['legacy_schema_aliasing'])

conn = engine.connect()

# Required for querying tables
metadata = MetaData(conn)

#Table to query
tbl = Table(TableName, metadata, autoload = True, schema = 'dbo')
# tbl.create(checkfirst = True)

#select all
sql = tbl.select()

#run sql code
result = conn.execute(sql)

df3 = pd.DataFrame(data = list(result), columns = result.keys())
df3.head()

In [ ]:
#Sau khi sử dụng connection thì chúng ta nhớ đóng lại connection để giải phóng memory và port.
%%script echo skipping

#close connection to free memory
conn.close()

In [ ]:
#Chúng ta cũng có thể thực thi các lệnh của SQL thông qua engine SQL mà chúng ta đã khởi tạo.
#Kết quả sẽ được truy vấn và tính toán trực tiếp từ server trả về như câu lệnh ta yêu cầu.

%%script echo skipping
#another way, use read_sql_query() function from pandas. This function use directly engine without initialize connection
pd.read_sql_query("SELECT TOP 5 * FROM WorkOrder", engine)

In [ ]:
#pyodbc

#Đây là package được thiết kế riêng để truy vấn trên những hệ cơ sở dữ liệu sử dụng kết nối ODBC của Microsoft.










In [ ]:
%%script echo skipping

import pandas.io.sql
import pyodbc

server = 'LAPTOPTCC-PC'
db = 'AdventureWorks'

#create connection
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' 
                      + DB['servername']
                      + ';DATABASE=' 
                      + DB['database'] 
                      + ';Trusted_Connection=yes')

In [ ]:
#Sau khi khởi tạo kết nối thì ta cũng có thể sử dụng các câu lệnh của SQL như thông thường để truy vấn và tính toán thông tin như bên dưới:
%%script echo skipping

#query db
sql = """SELECT * FROM WorkOrder"""

df4 = pandas.io.sql.read_sql(sql, conn)
df4.head()

In [ ]:
#Cursor cho SQL
#Cursor là một con trỏ dẫn tới một vùng nhớ mà lưu trữ dữ liệu.
#Sử dụng cursor sẽ giúp ta tiết kiệm bộ nhớ vì chúng ta không phải phân bổ bộ nhớ cho dữ liệu ngay mà chỉ sử dụng địa chỉ để trỏ tới dữ liệu.
#Cursor được sử dụng phổ biến trong SQL đặc biệt là trong các vòng for.
#Trong pyodbc chúng ta sẽ sử dụng cursor() trong vòng for để duyệt qua các dòng như bên dưới


In [ ]:
%%script echo skipping

import pyodbc

con = pyodbc.connect("DRIVER={SQL Server};"
                     "SERVER=LAPTOPTCC-PC;"
                     "DATASET=AdventureWorksDW2012;"
                     "Trusted_Connection=yes;")
cursor = con.cursor()

cursor.execute("SELECT TOP 10 * FROM AdventureWorksDW2012.dbo.DimCustomer")

for row in cursor.fetchall():
    print('row = ', row)